<a href="https://colab.research.google.com/github/Marcusreu1/House-prices/blob/main/House_prices%2C_Marcusreul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'house-prices-advanced-regression-techniques:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F5407%2F868283%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T010729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D355b2bd94034612a725c68265753488160fc65d9bfc652e1ee402f0a1bd22b70888e85b6f546e40e862af78be02d2aa7c2749f4f1c2ddb913f0ca4a83be9d3b353861467e15917a58c0142226d942daaffba0ec217184381868dff137b7f271fad02be7e5babdc1d668c75d88c4a24da1322d4273bb1610f0a1a9d580703289d0c885b8af4879523a1b124d31b648cef2180202cd6665816ac3986f755226c0ff8b932eecd336bfed5e226daac0fefdfe9c777d2953819e8d7e7a6cc56656fdf53ae434e31708991b130e9365ad65315ea3660fb809cbe44844ef1a34c54a99b2e5817088bcbad2f73a82f4d08451d8bd514be6b81e51b1a79eedeb4142d4ddc,house-prices:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4864911%2F8209510%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240427%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240427T010729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D663e074327ffd29f71945464af90e148e80053d5972ba109acb07de301236226d148b15489f60bde62846a928d102820203c2af1b5ca14f2078b5971741cd37b7fa5dcd445b3de6e6bc89d53ac245cc0c05bf32e95607b21392526077fdb16758d3c1955bd1ddf8324fba5fd50b5c883d696ea753788d9b56d26d1e131b3d6a14bcfdfe782cc7c3f1e90fc8b28865d8e0fabe407d92ff8255c222965c896f29c5bd026db06315d0de27bd37104ddc2651af13db42c46d0d9a2008ae99b11103f649a0ae8cf4fb86bf3f3b77feb2b58004d9f29fd6dd4a31f2718f92d9e92f3b0ef6591a20ebfc3f8fe3fa7f63bc1f136e1e4581d7765667dcff59558432df8f8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# importar librerias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# leer data

In [ ]:
df_train=pd.read_csv('/kaggle/input/house-prices/House prices/test house prices.csv')
df_test=pd.read_csv('/kaggle/input/house-prices/House prices/train house prices.csv')

# concatenar

In [ ]:
df=pd.concat([df_train,df_test])

# calidad y limpieza de datos

## vision general

In [ ]:
df.sample(5)

In [ ]:
df.shape

## tipo de datos

In [ ]:
df.info()

## valores unicos por columna

In [ ]:
columnas_categoricas = df.select_dtypes(include=['object']).columns

for column in columnas_categoricas:
  print(f"{column}: {df[column].unique()}")

## valores duplicados

In [ ]:
df.duplicated().sum()

## valores nulos

In [ ]:
pd.set_option('display.max_rows', None)
df.isnull().sum().sort_values(ascending = False)

### conclusiones
* dejar Saleprice para separar mas adelante
* PoolQC,MiscFeauture,Alley,Fence,MasVnrType, FirePlaceQU,LotFrontage tienen demasiados nulos ---> eliminar variables
* GarageYrBlt,GarageFinish,GarageQual,GarageCond,GarageType,BsmtExposure,BsmtCond,BsmtQual,BsmtFinType2,BsmtFinType1,MasVnrArea,  MSZoning,Functional,BsmtFullBath,Utilities,BsmtHalfBath,Electrical,TotalBsmtSF,BsmtUnfSF,KitchenQual,BsmtFinSF2,BsmtFinSF1,    SaleType,GarageCars,Exterior2nd,Exterior1st,GarageArea ---> imputarlos tras eda

In [ ]:
pd.reset_option('display.max_rows')
df.drop(columns=['PoolQC','MiscFeature','Alley','Fence','MasVnrType', 'FireplaceQu','LotFrontage'], inplace=True)
df

In [ ]:
pd.set_option('display.max_rows', None)
df.isnull().sum().sort_values(ascending = False)

# EDA

## variables categoricas

In [ ]:
def graficos_eda_categoricos(cat):

    #Calculamos el número de filas que necesitamos
    from math import ceil
    filas = ceil(cat.shape[1] / 2)

    #Definimos el gráfico
    f, ax = plt.subplots(nrows = filas, ncols = 2, figsize = (16, filas * 6))

    #Aplanamos para iterar por el gráfico como si fuera de 1 dimensión en lugar de 2
    ax = ax.flat

    #Creamos el bucle que va añadiendo gráficos
    for cada, variable in enumerate(cat):
        cat[variable].value_counts().plot.barh(ax = ax[cada])
        ax[cada].set_title(variable, fontsize = 12, fontweight = "bold")
        ax[cada].tick_params(labelsize = 12)


graficos_eda_categoricos(df.select_dtypes('O'))

In [ ]:
categorias = {'Categorica': [], 'Numerica': []}


for columna in df.columns:
    if df[columna].isnull().any():
        if df[columna].dtype == 'object':
            categorias['Categorica'].append(columna)
        elif df[columna].dtype in ['int64', 'float64']:
            categorias['Numerica'].append(columna)

# Imprimir las categorías con nulos
print('Categoricas:', categorias['Categorica'])
print('Numericas:', categorias['Numerica'])


### conclusiones
* Street,Heating,RoofMatl,Condition2,Utilities solo tiene valor en una variable ---> eliminar
* GarageCond ---> imputar por TA
* GarageFinish ---> imputar por Unf
* GarageQual ---> imputar por TA
* GarageType --->Attchd
* MSZoning ---> RL
* Exterior1st --->VinylSd
* Exterior 2nd ---> VinylSd
* BsmtQual ---> TA
* BsmtCond ---> TA
* BsmtExposure ---> No
* BsmtFinType1--->Unf
* BsmtFinType2--->Unf
* Electrical --->SBrkr
* KitchenQual --->TA
* Functional --->Typ
* SaleType--->WD


In [ ]:
# Eliminar columnas con un solo valor único
df.drop(columns=['Street', 'Heating', 'RoofMatl', 'Condition2', 'Utilities'], inplace=True)

# Imputar valores nulos con los valores especificados
df['GarageCond'] = df['GarageCond'].fillna('TA')
df['GarageFinish'] = df['GarageFinish'].fillna('Unf')
df['GarageQual'] = df['GarageQual'].fillna('TA')
df['GarageType'] = df['GarageType'].fillna('Attchd')
df['MSZoning'] = df['MSZoning'].fillna('RL')
df['Exterior1st'] = df['Exterior1st'].fillna('VinylSd')
df['Exterior2nd'] = df['Exterior2nd'].fillna('VinylSd')
df['BsmtQual'] = df['BsmtQual'].fillna('TA')
df['BsmtCond'] = df['BsmtCond'].fillna('TA')
df['BsmtExposure'] = df['BsmtExposure'].fillna('No')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('Unf')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('Unf')
df['Electrical'] = df['Electrical'].fillna('SBrkr')
df['KitchenQual'] = df['KitchenQual'].fillna('TA')
df['Functional'] = df['Functional'].fillna('Typ')
df['SaleType'] = df['SaleType'].fillna('WD')


#### comprobar que no quedo ningun nulo en las categoricas

In [ ]:
categorias = {'Categorica': [], 'Numerica': []}


for columna in df.columns:
    if df[columna].isnull().any():
        if df[columna].dtype == 'object':
            categorias['Categorica'].append(columna)
        elif df[columna].dtype in ['int64', 'float64']:
            categorias['Numerica'].append(columna)

# Imprimir las categorías con nulos
print('Categoricas:', categorias['Categorica'])
print('Numericas:', categorias['Numerica'])

## variables numericas


In [ ]:
def estadisticos_cont(num):
    # Calculamos describe
    estadisticos = num.describe().T
    # Añadimos la mediana
    estadisticos['median'] = num.median()
    # Añadimos la moda. Usamos apply para calcular la moda de cada columna por separado.
    estadisticos['moda'] = num.apply(lambda x: x.mode().get(0, None))

    # Reordenamos para que la mediana y la moda estén al lado de la media
    estadisticos = estadisticos.iloc[:, [0, 1, 8, 9, 2, 3, 4, 5, 6, 7]]
    # Lo devolvemos
    return(estadisticos)

# Llamamos a la función con el DataFrame 'df' que contiene solo columnas numéricas
estadisticos_cont(df.select_dtypes('number'))


In [ ]:
# Lista de columnas para imputar con la moda
columnas_para_imputar = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                         'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

# Imputar la moda en las columnas especificadas
for columna in columnas_para_imputar:
    moda = df[columna].mode()[0]
    df.loc[df[columna].isnull(), columna] = moda





In [ ]:
categorias = {'Categorica': [], 'Numerica': []}


for columna in df.columns:
    if df[columna].isnull().any():
        if df[columna].dtype == 'object':
            categorias['Categorica'].append(columna)
        elif df[columna].dtype in ['int64', 'float64']:
            categorias['Numerica'].append(columna)

# Imprimir las categorías con nulos
print('Categoricas:', categorias['Categorica'])
print('Numericas:', categorias['Numerica'])

In [ ]:
df.isnull().sum().sort_values(ascending = False)

# Modelo de machine learning

## PREPARACIÓN DE LOS DATOS PARA LA MODELIZACIÓN

### Transformar todas las variables categóricas a númericas

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#Categóricas
cat = df.select_dtypes('O')

#Instanciamos
ohe = OneHotEncoder(sparse_output = False)

#Entrenamos
ohe.fit(cat)

#Aplicamos
cat_ohe = ohe.transform(cat)

#Ponemos los nombres
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out(input_features = cat.columns)).reset_index(drop = True)

In [ ]:
pd.reset_option('display.max_rows')
cat_ohe

### dataframe final
Seleccionamos las variables numéricas para poder juntarlas a las cat_hoe

In [ ]:
num = df.select_dtypes('number').reset_index(drop = True)

Las juntamos todas en el dataframe final

In [ ]:
df_ml = pd.concat([cat_ohe,num], axis = 1)
df_ml

## separacion de dataframe

In [ ]:
df_train= df_ml[df_ml['SalePrice'].notnull()]
df_test=df_ml[df_ml['SalePrice'].isnull()]
df_test = df_test.drop(columns='SalePrice', errors='ignore')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## diseño de la modelizacion

### separacion de predictoras y target

In [ ]:
X = df_train.drop(columns='SalePrice')  # Características
y = df_train['SalePrice']               # Variable objetivo

### Separación train y testSeparación train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## entrenamiento de modelo sobre train

### creacion del modelo

In [ ]:
from sklearn.linear_model import LinearRegression
regresion=LinearRegression()
regresion.get_params()

### hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parámetros para la regresión lineal
params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, -1],
    'positive': [True, False]
}

# Crear la rejilla de búsqueda
rejilla = GridSearchCV(regresion,
                       params,
                       scoring="neg_mean_squared_error",
                       cv=10)

In [ ]:
rejilla.fit(X_train, y_train)

In [ ]:
model= rejilla.best_estimator_

# predecir

In [ ]:
# Realizar predicciones
predicciones = model.predict(df_test)

# Agregar las predicciones al DataFrame
df_test['SalePrice'] = predicciones

In [ ]:
df_test.head()

In [ ]:
df= df_test[['Id','SalePrice']]
df.head()

In [ ]:
df.to_csv('submission.csv', index=False)